In [1]:
# --- Setup: Imports and Path Adjustments ---
import pandas as pd
import sys
import os

# Adjust sys.path to allow imports from src/
current_notebook_dir = os.getcwd()  # Use current working directory for Jupyter notebooks
project_root_dir = os.path.join(current_notebook_dir, '..')  # Go up to 'solar-challenge-week1/'
sys.path.insert(0, project_root_dir)

# Import functions from your src modules
import scripts.data_cleaning as dc
import scripts.data_profiling as dp
import scripts.eda_plots as ep

# Define country name for consistent naming
COUNTRY = "Benin"
DATA_FILE = "benin-malanville.csv"

# **1. Load Raw Data**

We begin by loading the raw data for Benin. The dataset is expected to be in the `data/` directory. If the file is missing, an error will be raised.

In [2]:
# --- 1. Load Raw Data ---
print(f"--- Loading Raw Data for {COUNTRY} ---")
data_path = os.path.join(project_root_dir, 'data', DATA_FILE)

try:
    df_raw = pd.read_csv(data_path)
    print(f"Successfully loaded {DATA_FILE}.")
except FileNotFoundError:
    print(f"Error: Raw data file not found at {data_path}. Please check the path and file existence.")
    sys.exit("Data file not found.")

print("\n**Raw Data Head:**")
display(df_raw.head())
print(f"\n**Raw Data Shape:** {df_raw.shape}")

--- Loading Raw Data for Benin ---
Successfully loaded benin-malanville.csv.

**Raw Data Head:**


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-08-09 00:01,-1.2,-0.2,-1.1,0.0,0.0,26.2,93.4,0.0,0.4,0.1,122.1,0.0,998,0,0.0,26.3,26.2,NaN
1,2021-08-09 00:02,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.6,0.0,0.0,0.0,0.0,0.0,998,0,0.0,26.3,26.2,NaN
2,2021-08-09 00:03,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.7,0.3,1.1,0.5,124.6,1.5,997,0,0.0,26.4,26.2,NaN
3,2021-08-09 00:04,-1.1,-0.1,-1.0,0.0,0.0,26.2,93.3,0.2,0.7,0.4,120.3,1.3,997,0,0.0,26.4,26.3,NaN
4,2021-08-09 00:05,-1.0,-0.1,-1.0,0.0,0.0,26.2,93.3,0.1,0.7,0.3,113.2,1.0,997,0,0.0,26.4,26.3,NaN



**Raw Data Shape:** (525600, 19)


# **2. Initial Data Profiling**

Before cleaning, we perform an initial data profiling to understand the raw state of the dataset. This includes summary statistics and a missing value report.

In [3]:
# --- 2. Initial Data Profiling (Raw Data) ---
print("\n--- Initial Data Profiling (Raw Data) ---")
display(dp.get_summary_statistics(df_raw.select_dtypes(include=['number'])))
dp.print_missing_value_report(df_raw, threshold=5)  # List columns with >5% nulls


--- Initial Data Profiling (Raw Data) ---

--- Summary Statistics ---


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,240.559452,167.187516,115.358961,236.589496,228.883576,28.179683,54.487969,2.121113,2.809195,0.473390,153.435172,8.582407,994.197199,0.000923,0.001905,35.246026,32.471736,NaN
std,331.131327,261.710501,158.691074,326.894859,316.536515,5.924297,28.073069,1.603466,2.029120,0.273395,102.332842,6.385864,2.474993,0.030363,0.037115,14.807258,12.348743,NaN
min,-12.900000,-7.800000,-12.600000,0.000000,0.000000,11.000000,2.100000,0.000000,0.000000,0.000000,0.000000,0.000000,985.000000,0.000000,0.000000,9.000000,8.100000,NaN
25%,-2.000000,-0.500000,-2.100000,0.000000,0.000000,24.200000,28.800000,1.000000,1.300000,0.400000,59.000000,3.700000,993.000000,0.000000,0.000000,24.200000,23.600000,NaN
50%,1.800000,-0.100000,1.600000,4.500000,4.300000,28.000000,55.100000,1.900000,2.600000,0.500000,181.000000,8.600000,994.000000,0.000000,0.000000,30.000000,28.900000,NaN
75%,483.400000,314.200000,216.300000,463.700000,447.900000,32.300000,80.100000,3.100000,4.100000,0.600000,235.100000,12.300000,996.000000,0.000000,0.000000,46.900000,41.500000,NaN
max,1413.000000,952.300000,759.200000,1342.300000,1342.300000,43.800000,100.000000,19.500000,26.600000,4.200000,360.000000,99.400000,1003.000000,1.000000,2.500000,81.000000,72.500000,NaN



--- Missing Value Report ---
Missing values by column:
          Missing Count  Missing Percentage
Comments         525600               100.0

Columns with more than 5% missing values:
          Missing Count  Missing Percentage
Comments         525600               100.0


# **3. Clean Data**

The raw data is cleaned using the `clean_data` function from `src/data_cleaning.py`. This process includes handling missing values, removing outliers, and ensuring data integrity.

In [4]:
# --- 3. Clean Data ---
print(f"\n--- Cleaning Data for {COUNTRY} using src/data_cleaning.py ---")
df_cleaned = dc.clean_data(df_raw.copy(), country_name=COUNTRY)

print("\n**Cleaned Data Head:**")
display(df_cleaned.head())
print(f"\n**Cleaned Data Shape:** {df_cleaned.shape}")


--- Cleaning Data for Benin using src/data_cleaning.py ---

--- Cleaning Data for Benin ---

  Enforcing Physical Constraints (Setting impossible values to NaN)...
    Corrected 258847 negative values in 'GHI'.
    Corrected 275987 negative values in 'DNI'.
    Corrected 259182 negative values in 'DHI'.
  Total physically impossible values corrected: 794016

  Dropped 278722 rows due to missing values in critical columns: GHI, DNI, DHI, ModA, ModB.

  Imputing remaining numerical missing values with median...


c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\.env\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\data_cleaning.py:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Comments'].fillna('No Comment', inplace=True)
c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\

  No further numerical missing values required imputation.
  Filled 246878 missing 'Comments' with 'No Comment'.

  Detecting potential statistical outliers (Z-score > 3) for reporting...
  Potential statistical outliers detected (Z-score > 3):
    - DHI: 168 outliers
    - Tamb: 736 outliers
    - WS: 614 outliers
    - WSgust: 604 outliers
    - BP: 353 outliers
    - Precipitation: 640 outliers
    - TModA: 9 outliers
    - TModB: 74 outliers
  Note: These values are flagged for awareness but not automatically removed or capped by this function.

--- Cleaning Summary for Benin ---
  Initial rows: 525600
  Final rows after cleaning: 246878
  Total rows removed/adjusted: 278722

  Missing values after cleaning:
Series([], dtype: int64)

Cleaned data saved to: c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\scripts\..\data\benin_clean.csv

**Cleaned Data Head:**


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
Timestamp,,,,,,,,,,,,,,,,,,
2021-08-09 06:54:00,16.7,0.0,16.5,16.1,16.3,24.2,98.8,0.0,0.0,0.0,0.0,0.0,997,0,0.0,24.2,23.7,No Comment
2021-08-09 06:55:00,18.2,0.1,18.0,17.4,17.6,24.2,98.8,0.0,0.0,0.0,0.0,0.0,997,0,0.0,24.3,23.8,No Comment
2021-08-09 06:56:00,19.7,0.3,19.5,18.7,18.9,24.2,98.8,0.0,0.0,0.0,0.0,0.0,997,0,0.0,24.3,23.9,No Comment
2021-08-09 06:57:00,21.1,0.6,20.9,19.9,20.1,24.2,98.9,0.0,0.0,0.0,0.0,0.0,997,0,0.0,24.4,23.9,No Comment
2021-08-09 06:58:00,22.5,1.1,22.2,21.1,21.3,24.2,98.9,0.0,0.0,0.0,0.0,0.0,997,0,0.0,24.5,24.0,No Comment



**Cleaned Data Shape:** (246878, 18)


# **4. Post-Cleaning Data Profiling**

After cleaning, we re-profile the data to verify the effectiveness of the cleaning process. This includes updated summary statistics and a missing value report.# **4. Post-Cleaning Data Profiling**

After cleaning, we re-profile the data to verify the effectiveness of the cleaning process. This includes updated summary statistics and a missing value report.

In [5]:
# --- 4. Post-Cleaning Data Profiling ---
print(f"\n--- Post-Cleaning Data Profiling for {COUNTRY} ---")
print("\n**Summary Statistics (Cleaned Data):**")
display(dp.get_summary_statistics(df_cleaned.select_dtypes(include=['number'])))

print("\n**Missing Value Report (Cleaned Data):**")
dp.print_missing_value_report(df_cleaned, threshold=0)  # Check if any nulls remain


--- Post-Cleaning Data Profiling for TOGO ---

**Summary Statistics (Cleaned Data):**

--- Summary Statistics ---


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB
count,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000,268215.000000
mean,454.081218,296.409415,228.166180,443.109538,430.224900,29.868625,50.954453,2.897560,3.951753,0.650577,171.400294,13.115981,975.956483,0.001048,0.001600,39.984648,42.444731
std,319.096010,283.534417,150.064955,318.155810,308.539523,4.825459,27.871583,1.474324,1.850927,0.265139,89.270858,5.635309,2.341965,0.032351,0.029865,10.425510,12.060518
min,0.000000,0.000000,0.000000,0.000000,0.000000,14.900000,3.300000,0.000000,0.000000,0.000000,0.000000,0.000000,968.000000,0.000000,0.000000,13.700000,13.400000
25%,156.500000,7.000000,112.400000,144.200000,140.700000,26.400000,22.800000,1.900000,2.600000,0.500000,83.200000,10.000000,975.000000,0.000000,0.000000,31.900000,33.000000
50%,430.300000,233.800000,211.400000,410.800000,399.800000,29.900000,55.200000,2.800000,3.900000,0.600000,202.800000,12.900000,976.000000,0.000000,0.000000,40.300000,42.600000
75%,743.900000,548.000000,330.300000,733.400000,710.400000,33.500000,74.600000,3.800000,5.200000,0.800000,240.700000,16.000000,978.000000,0.000000,0.000000,47.800000,51.400000
max,1424.000000,1004.500000,805.700000,1380.000000,1367.000000,41.400000,99.800000,16.100000,22.900000,4.400000,360.000000,86.900000,983.000000,1.000000,2.300000,70.400000,94.600000



**Missing Value Report (Cleaned Data):**

--- Missing Value Report ---
No missing values found in the DataFrame.


# **5. Time Series Analysis**

We analyze the cleaned data over time, focusing on key variables such as GHI, DNI, DHI, and Tamb. This includes plotting time series and average daily/monthly profiles.

In [6]:
# --- 5. Time Series Analysis ---
print(f"\n--- Time Series Analysis for {COUNTRY} ---")

# GHI, DNI, DHI over Time
ep.plot_time_series(df_cleaned, ['GHI', 'DNI', 'DHI'], COUNTRY, title_suffix="Irradiance")

# Tamb over Time
ep.plot_time_series(df_cleaned, 'Tamb', COUNTRY, title_suffix="Ambient Temperature")

# Average Daily Profiles (hourly)
ep.plot_daily_average_profile(df_cleaned, ['GHI', 'Tamb'], COUNTRY)

# Average Monthly Profiles
ep.plot_monthly_average_profile(df_cleaned, ['GHI', 'Tamb'], COUNTRY)


--- Time Series Analysis for TOGO ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_dni_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_dhi_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_tamb_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_daily_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_tamb_daily_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_monthly_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_tamb_monthly_profile.png


### **6. Cleaning Impact Analysis**

In [7]:
print(f"\n--- Cleaning Impact Analysis for {COUNTRY} ---")
ep.plot_cleaning_impact(df_cleaned, 'ModA', 'ModB', COUNTRY)


--- Cleaning Impact Analysis for TOGO ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_cleaning_impact.png


### **7. Correlation & Relationship Analysis**

In [8]:
print(f"\n--- Correlation & Relationship Analysis for {COUNTRY} ---")

# Heatmap
ep.plot_correlation_heatmap(df_cleaned, ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'Precipitation'], COUNTRY)

# Scatter plots
ep.plot_scatter(df_cleaned, 'WS', 'GHI', COUNTRY, title_suffix="(Wind Speed vs GHI)")
ep.plot_scatter(df_cleaned, 'WSgust', 'GHI', COUNTRY, title_suffix="(Wind Gust vs GHI)")
ep.plot_scatter(df_cleaned, 'RH', 'Tamb', COUNTRY, title_suffix="(Relative Humidity vs Ambient Temperature)")
ep.plot_scatter(df_cleaned, 'RH', 'GHI', COUNTRY, title_suffix="(Relative Humidity vs GHI)")


--- Correlation & Relationship Analysis for TOGO ---
  Saved matplotlib plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_correlation_heatmap.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_vs_ws_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_vs_wsgust_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_tamb_vs_rh_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_vs_rh_scatter.png


### **8. Wind & Distribution Analysis**

In [9]:
print(f"\n--- Wind & Distribution Analysis for {COUNTRY} ---")

# Wind Rose
ep.plot_wind_rose(df_cleaned, 'WS', 'WD', COUNTRY)

# Histograms
ep.plot_distribution(df_cleaned, 'GHI', COUNTRY, hist_type='histogram')
ep.plot_distribution(df_cleaned, 'WS', COUNTRY, hist_type='histogram')
ep.plot_distribution(df_cleaned, 'RH', COUNTRY, hist_type='histogram')


--- Wind & Distribution Analysis for TOGO ---


c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\eda_plots.py:233: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_wind_rose.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_distribution.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ws_distribution.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_rh_distribution.png


### **9. Bubble Chart**

In [10]:
print(f"\n--- Bubble Chart for {COUNTRY} ---")

# GHI vs. Tamb with bubble size = RH
ep.plot_bubble_chart(df_cleaned, 'Tamb', 'GHI', 'RH', COUNTRY)

# GHI vs. Tamb with bubble size = BP (if BP is relevant as a bubble size)
ep.plot_bubble_chart(df_cleaned, 'Tamb', 'GHI', 'BP', COUNTRY)


--- Bubble Chart for TOGO ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_vs_tamb_rh_bubble.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\togo_ghi_vs_tamb_bp_bubble.png
